In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Fairness_NLP/RULE_BASED')


In [ ]:
!ls

depression_dataset_reddit_cleaned.csv  RuleBasedSystem_DepressionAnalysis.ipynb
Redditpost_adj_embeddings.ipynb


In [ ]:
import torch
print(torch.__version__)

2.5.1+cu121


In [3]:
import csv
from collections import defaultdict, Counter

import torch
import numpy as np
import spacy
import pickle

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load spaCy's English model for POS tagging
nlp = spacy.load("en_core_web_sm")

# Load BERT model and tokenizer, then move model to the selected device
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased').to(device)

from transformers import AutoTokenizer, AutoModel
access_token = "TOKEN"

tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base", token=access_token)
model = AutoModel.from_pretrained("mental/mental-roberta-base", token=access_token).to(device)
# Function to get the embedding of a unigram in the context of a full sentence
def get_unigram_embedding_in_context(sentence, unigram):
    # Tokenize and prepare input, moving inputs to the GPU if available
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.squeeze(0)  # Shape: [seq_length, hidden_size]

    # Tokenize unigram separately to locate its position within the sentence
    unigram_tokens = tokenizer.tokenize(unigram)
    unigram_token_ids = tokenizer.convert_tokens_to_ids(unigram_tokens)

    # Locate unigram within the sentence by matching token IDs
    sentence_token_ids = inputs['input_ids'].squeeze().tolist()
    for i in range(len(sentence_token_ids) - len(unigram_token_ids) + 1):
        if sentence_token_ids[i:i + len(unigram_token_ids)] == unigram_token_ids:
            # Move embedding to CPU and detach to get the numpy array
            unigram_embedding = embeddings[i:i + len(unigram_token_ids)].mean(dim=0).detach().cpu().numpy()
            return unigram_embedding

    # If unigram is not found, return a zero embedding
    return np.zeros(embeddings.size(-1))

# Step 1: Read sentences and filter unigrams that are adjectives
unigram_embeddings = defaultdict(list)
unigram_counts = Counter()

# Replace 'your_file.csv' with your CSV file path
with open('/content/drive/MyDrive/Fairness_NLP/RULE_BASED/depression_dataset_reddit_cleaned.csv', 'r') as file:
    c = 0
    reader = csv.reader(file)
    for row in reader:
        sentence = row[0].strip()

        # POS tagging using spaCy to find adjectives
        doc = nlp(sentence)
        for token in doc:
            if token.pos_ in {"ADJ"}:  # Check if POS is adjective
                unigram = token.text
                unigram_counts[unigram] += 1  # Count the occurrence of each unigram
                unigram_embedding = get_unigram_embedding_in_context(sentence, unigram)
                unigram_embeddings[unigram].append(unigram_embedding)
        c += 1


# Display total number of unique unigrams and their individual counts
print(f"Total unique unigrams (adjectives): {len(unigram_counts)}")
print("Individual unigram counts:")
monitor = {}
for unigram, count in unigram_counts.items():
    monitor[unigram] = count

# Step 2: Calculate average embeddings for unigrams
average_unigram_embeddings = {}
for unigram, embeddings in unigram_embeddings.items():
    if len(embeddings) > 1:
        # Average embeddings if unigram appears more than once
        avg_embedding = np.mean(embeddings, axis=0)
    else:
        avg_embedding = embeddings[0]
    average_unigram_embeddings[unigram] = avg_embedding

# Output results: save to file
with open('adjective_embeddings.pkl', 'wb') as file:
    pickle.dump(average_unigram_embeddings, file)


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total unique unigrams (adjectives): 3143
Individual unigram counts:
